In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Load the entire JSON file
file_path = "data/dynamic_log_icra2024.json"
with open(file_path, 'r') as file:
    data = json.load(file)

# Create DataFrame
df = pd.json_normalize(data)

# Calculate ratios
df['Discoverable/Total Ratio'] = df['parameters.problem.n_discoverable'] / df['parameters.problem.n_total']
df['Visited/Total Ratio'] = df['result.n_visited'] / df['parameters.problem.n_total']
df['Total Time'] = df['result.solution_segments'].apply(lambda x: sum(seg['time'] for seg in x))
df['Total Path Length'] = df['result.solution_segments'].apply(lambda x: sum(seg['path_length'] for seg in x))

# Find the computation time and path length when Discoverable/Total Ratio is 0 (our references)
reference_time = df.loc[df['Discoverable/Total Ratio'] == 0, 'Total Time'].values[0]
reference_length = df.loc[df['Discoverable/Total Ratio'] == 0, 'Total Path Length'].values[0]

# Compute the time and length ratios to the time and length with 0 discoverable
df['Time Ratio to 0 Discoverable'] = df['Total Time'] / reference_time
df['Length Ratio to 0 Discoverable'] = df['Total Path Length'] / reference_length

# Sort DataFrame by 'Discoverable/Total Ratio' and 'parameters.planner'
df.sort_values(['Discoverable/Total Ratio', 'parameters.planner'], inplace=True)

# Separate DataFrame by planner type
df_fre = df[df['parameters.planner'] == 'dynamic_planner_fre']
df_lci = df[df['parameters.planner'] == 'dynamic_planner_lci']

# Metrics to plot
metrics = ['Time Ratio to 0 Discoverable', 'Length Ratio to 0 Discoverable', 'Visited/Total Ratio']

# Create a line plot for each metric for both planner types
for metric in metrics:
    plt.figure(figsize=(10, 6))
    plt.plot(df_fre['Discoverable/Total Ratio'], df_fre[metric], marker='o', label='FRE ' + metric)
    plt.plot(df_lci['Discoverable/Total Ratio'], df_lci[metric], marker='x', label='LCI ' + metric)
    plt.ylim(bottom=0)
    plt.xlabel('Discoverable/Total Ratio')
    plt.ylabel(metric)
    plt.title(metric + ' vs Discoverable/Total Ratio')
    plt.legend()
    plt.grid(True)
    plt.show()
